**Case study 4: Data-driven insights for enhancing hydrogen production forecasts using advanced analytics**

In [ ]:
import pandas as pd  
import numpy as np  
import xgboost as xgb  
from sklearn.model_selection import train_test_split, KFold, cross_val_score  
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, make_scorer  
import matplotlib.pyplot as plt  

# Load Data  
df=pd.read_csv('Hydrogen_Production_Data.csv')  

# Features and target  
features = ['Renewable_Energy_Input_MWh', 'Temperature_C', 'Operating_Pressure_bar']  
target = 'Hydrogen_Production_Tons'  
X = df[features]  
y = df[target]  

# Split the data into train and test sets (80-20 split)  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  

# Model Setup  
model = xgb.XGBRegressor(  
    colsample_bytree=0.6,  
    learning_rate=0.1,  
    max_depth=4,  
    reg_alpha=0.8,  
    reg_lambda=0.8,  
    subsample=0.6,  
    objective='reg:squarederror',  
    n_estimators=500,  
    seed=42 )  

# Fit model  
model.fit(X_train, y_train)  

# Predict on train and test sets  
y_train_pred = model.predict(X_train)  
y_test_pred = model.predict(X_test)  

# Calculate evaluation metrics for train set  
train_mse = mean_squared_error(y_train, y_train_pred)  
train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)  
train_mae = mean_absolute_error(y_train, y_train_pred)  
train_r2 = r2_score(y_train, y_train_pred)  

# Calculate evaluation metrics for test set  
test_mse = mean_squared_error(y_test, y_test_pred)  
test_rmse = mean_squared_error(y_test, y_test_pred, squared=False)  
test_mae = mean_absolute_error(y_test, y_test_pred)  
test_r2 = r2_score(y_test, y_test_pred)  

# Print metrics for train  
print("\nTrain Metrics:")  
print(f"R2: {train_r2:.4f}, MSE: {train_mse:.4f}, RMSE: {train_rmse:.4f}, MAE: {train_mae:.4f}")  

# Print metrics for test  
print("\nTest Metrics:")  
print(f"R2: {test_r2:.4f}, MSE: {test_mse:.4f}, RMSE: {test_rmse:.4f}, MAE: {test_mae:.4f}")  

# Cross-Validation for Train R2  
kf_train = KFold(n_splits=5, shuffle=True, random_state=42)  
cv_r2_scores_train = cross_val_score(model, X_train, y_train, cv=kf_train, scoring=make_scorer(r2_score))  
print("\nCross-Validation R2 Scores (Train):")  
print(f"Mean R2: {cv_r2_scores_train.mean():.4f} ± {cv_r2_scores_train.std():.4f}")  

# Cross-Validation for Test R2  
kf_test = KFold(n_splits=5, shuffle=True, random_state=42)  
cv_r2_scores_test = cross_val_score(model, X_test, y_test, cv=kf_test, scoring=make_scorer(r2_score))  
print("\nCross-Validation R2 Scores (Test):")  
print(f"Mean R2: {cv_r2_scores_test.mean():.4f} ± {cv_r2_scores_test.std():.4f}")  

# Plot Feature Importance  
plt.figure(figsize=(10, 8))  
ax = xgb.plot_importance(model, importance_type='weight', max_num_features=3, grid=False)  
ax.set_title('Feature Importance', fontsize=14)  
ax.set_xlabel('F-Score', fontsize=12)  
ax.set_ylabel('Features', fontsize=12)  

# Customize bar colors  
for bar in ax.patches:  
    bar.set_color('crimson')  

# Remove the text labels on the bars  
for text in ax.texts:  
    text.set_visible(False)  

plt.show()  

# Performance Plot  
plt.figure(figsize=(16, 6))  
plt.scatter(df['Date'], df['Hydrogen_Production_Tons'], label='Actual Hydrogen Production', color='crimson', s=10, alpha=0.7)  
plt.plot(df['Date'], model.predict(X), label='Model Prediction', color='#1E90FF', linestyle='-', linewidth=2, alpha=0.8)  
plt.xlabel('Date', fontsize=12)  
plt.ylabel('Hydrogen Production (Tons)', fontsize=12)  
plt.title('Model Performance: Actual vs Predicted Hydrogen Production', fontsize=14)  
plt.legend(loc='best', fontsize=10)  
plt.ylim(90, 170)    
plt.show()  

# Train Data Cross Plot  
plt.figure(figsize=(14, 6))  
plt.subplot(1, 2, 1)  
plt.scatter(y_train, y_train_pred, color='#1E90FF', edgecolor='k', alpha=0.6)  
plt.plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'k--', lw=2)  
plt.xlabel('Actual Hydrogen Production (Tons)', fontsize=12)  
plt.ylabel('Predicted Hydrogen Production (Tons)', fontsize=12)  
plt.title(f'XGBoost_Train Data: R²={train_r2:.4f}', fontsize=14)  
plt.xlim(90, 170)   
plt.ylim(90, 170)  
plt.legend(['Train Data', 'Y=X'], fontsize='medium')  

# Test Data Cross Plot  
plt.subplot(1, 2, 2)  
plt.scatter(y_test, y_test_pred, color='crimson', edgecolor='k', alpha=0.6)  
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)  
plt.xlabel('Actual Hydrogen Production (Tons)', fontsize=12)  
plt.ylabel('Predicted Hydrogen Production (Tons)', fontsize=12)  
plt.title(f'XGBoost_Test Data: R²={test_r2:.4f}', fontsize=14)  
plt.xlim(90, 170)   
plt.ylim(90, 170)  
plt.legend(['Test Data', 'Y=X'], fontsize='medium')  

plt.tight_layout()  
plt.show()